Script to optimize the fluorescence from single atom imaging.\
We use M-LOOP to feedback the information between the experiment and this computer, which controls the parameters to optimize.\
The input parameters are produced by M-LOOP in the file "exp_input.txt". This file is read by this script and the parameters passed as variables. Then the program is sent to OPX for execution: it waits the trigger from Logicbox. Then an image is produced, saved in "Whitlock-PC", has to be passed to this computer. Once it's in this computer, we evaluate the cost and produce the file "exp_output" which is read by M-LOOP: it applies the opimization algorithm specified in the configuration file and produces a new "exp_input" file. The loop is closed.

So we have two files that work together: one for control and a second one that reads the received image and evaluates the cost.

In [ ]:
from qm.qua import *
from qm import QuantumMachinesManager, SimulationConfig
from quam.components import BasicQuAM, SingleChannel
from quam.components.pulses import SquarePulse
from qualang_tools.units import unit
from QuAM_utilities import *

import os
import matplotlib.pyplot as plt
from matplotlib import patches
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler, PatternMatchingEventHandler
import numpy as np
from numpy import array
from astropy.io import fits

u = unit()

2024-12-06 14:27:52,206 - qm - INFO     - Starting session: a1716dce-5493-4b92-a90b-195097014023


In [2]:
# silence the warnings?
import warnings
warnings.filterwarnings("ignore")

from IPython.display import clear_output

In [3]:
ip_address = '130.79.148.167'
qmm = QuantumMachinesManager(host=ip_address)

2024-12-06 14:27:55,264 - qm - INFO     - Performing health check
2024-12-06 14:27:55,276 - qm - INFO     - Health check passed


Configure the machine: define the channels and associate the pulses

In [4]:
machine = BasicQuAM()

# AOM RFs
cool_double_rf = 200*u.MHz
repump_double_rf = 200*u.MHz

# TWEEZER LOADING
tweez_central_freq = 74.9*u.MHz
tweez_difference_freq = 15*u.MHz
tweez_low_freq = tweez_central_freq - tweez_difference_freq/2
tweez_high_freq = tweez_central_freq + tweez_difference_freq/2

tweez_ramp_amplitude = 0.07
tweez_ramp_time = 100*u.ms #50*u.us #
tweez_plateau_amplitude = tweez_ramp_amplitude
tweez_plateau_time = 150.15*u.ms #50*u.us #

# GAP 1
gap1_time = 5*u.ms #200*u.ns#

# COLLISION
coll_time = 100*u.ms #100*u.us #
tweez_square_amplitude_coll = 0.1
repump_square_amplitude_coll_double = 0.38 # fixed
cool_square_amplitude_coll_double = 0.38 # fixed
coll_modulation_frequency = 1.7*u.MHz
tweez_duty_cycle = 0.4 # high fraction for tweezer
rc_duty_cycle = 0.3 # high fraction for repumper and cooler
rc_delay_fraction = 0.05 # delay for repumper and cooler (fraction of period) 
                        # the sum of the repumper/cooler duty cycles must be less than 1

# GAP 2
gap2_time = 5*u.ms #20*u.us #
# look if after this we have an intrinsic delay of 35 ns?

# IMAGING
# the duty cycle parameters are the same as for collision
imag_time = 200*u.ms #80*u.us #
tweez_square_amplitude_imag = 0.15
repump_square_amplitude_imag_double = 0.38 # fixed
cool_square_amplitude_imag_double = 0.38 # fixed

#coll_time_single_pass = gap1_time + coll_time
#imag_time_single_pass = gap2_time + imag_time

# DELAY FOR IMAGING
delay_imaging_time = 350.15*u.ms #20*u.us #

# FOR CONTINUOUSLY DRIVEN AOMs: cooler/repumper single pass & two spots tweezers
# this is for amplitude modulation
continuous_time = 50*u.ms
continuous_time_long_pulse = tweez_ramp_time + tweez_plateau_time + gap1_time + coll_time + gap2_time + imag_time + delay_imaging_time + imag_time + 50*u.ms
# continuous_time_long_pulse = 1*u.s

# collision
repump_square_amplitude_coll_single = 0.07 #0.05
cool_square_amplitude_coll_single = 0.1 #0.07

# imaging
repump_square_amplitude_imag_single = 0.09
cool_square_amplitude_imag_single = 0.08
spot_amplitude1 = 0.38 # fixed
spot_amplitude2 = 0.38 # fixed 

In [5]:
# additional dependent variables useful for pulse definition
# NB: the time durations must be multiples of the clock cycle!

# these are not used in the function, but are printed for debug
tweez_ramp_step_time = 40*u.ns
n_steps = round(tweez_ramp_time/tweez_ramp_step_time)
plateau_intermediate_duration = 10*u.us # prone to bug: this must be an integer of plateau_duration # INSIDE THE FUNCTION, AS AN ARGUMENT?
plateau_num_portions = round(tweez_plateau_time/plateau_intermediate_duration) #INSIDE THE FUNCTION

# these are in seconds, multiply by u.s before passing as arguments
coll_modulation_period = 1/coll_modulation_frequency
tweez_high_time = 1/coll_modulation_frequency*tweez_duty_cycle
tweez_low_time = 1/coll_modulation_frequency*(1-tweez_duty_cycle)
rc_high_time = 1/coll_modulation_frequency*rc_duty_cycle
rc_delay_time = 1/coll_modulation_frequency*rc_delay_fraction
rc_low_time = coll_modulation_period-rc_delay_time-rc_high_time
number_of_periods_collision = round(coll_time/(coll_modulation_period*u.s)) # used as argument

imag_modulation_period = coll_modulation_period

number_of_periods_imaging = round(imag_time/(imag_modulation_period*u.s)) # used as argument

# for debugging
print(f"\n Number of steps for the tweezer ramp: {n_steps}, each of {tweez_ramp_step_time} ns")
print(f"Number of plateau portions: {plateau_num_portions}")
print("Number of collision periods:", number_of_periods_collision)
print("Number of imaging periods:", number_of_periods_imaging, "\n")
print(f"Low duration for tweezer: {round(tweez_low_time*u.s/4)*4}ns")
print(f"High duration for tweezer: {round(tweez_high_time*u.s/4)*4}ns")
print(f"Total: {round(tweez_low_time*1e9/4)*4 + round(tweez_high_time*u.s/4)*4}ns \n")
print(f"Delay duration for repumper/cooler: {round(rc_delay_time*u.s/4)*4}ns")
print(f"High duration for repumper/cooler: {round(rc_high_time*u.s/4)*4}ns")
print(f"Low duration for repumper/cooler: {round(rc_low_time*u.s/4)*4}ns")
print(f"Total: {round(rc_delay_time*u.s/4)*4 + round(rc_low_time*u.s/4)*4 + round(rc_high_time*u.s/4)*4}ns \n")



 Number of steps for the tweezer ramp: 2500000, each of 40.0 ns
Number of plateau portions: 15015
Number of collision periods: 170000
Number of imaging periods: 340000 

Low duration for tweezer: 352ns
High duration for tweezer: 236ns
Total: 588ns 

Delay duration for repumper/cooler: 28ns
High duration for repumper/cooler: 176ns
Low duration for repumper/cooler: 384ns
Total: 588ns 



Configure the machine: define the channels and add the pulses

In [6]:
machine.channels['AOM_single_repumper'] = AOM_single_repumper = SingleChannel(opx_output=('con1', 1), intermediate_frequency=200e6)
machine.channels['AOM_double_repumper'] = AOM_double_repumper = SingleChannel(opx_output=('con1', 2), intermediate_frequency=repump_double_rf)
machine.channels['AOM_single_cooler'] = AOM_single_cooler = SingleChannel(opx_output=('con1', 3), intermediate_frequency=200e6)
machine.channels['AOM_double_cooler'] = AOM_double_cooler = SingleChannel(opx_output=('con1', 4), intermediate_frequency=cool_double_rf)
machine.channels['AOM_tweez_mod'] = AOM_tweez_mod = SingleChannel(opx_output=('con1', 5), intermediate_frequency=89.161e6) # for tweezer modulation
machine.channels['AOM_tweez_spots1'] = AOM_tweez_spots1 = SingleChannel(opx_output=('con1', 6), intermediate_frequency=tweez_high_freq) # first component for generating two spots
machine.channels['AOM_tweez_spots2'] = AOM_tweez_spots2 = SingleChannel(opx_output=('con1', 6), intermediate_frequency=tweez_low_freq) # second component for generating two spots

AOM_single_repumper.operations['repump_continuous_coll'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=repump_square_amplitude_coll_single)
AOM_single_repumper.operations['repump_continuous_imag'] = repump_continuous_imag = SquarePulse(length=round(continuous_time/4)*4, amplitude=repump_square_amplitude_imag_single)
AOM_single_cooler.operations['cool_continuous_coll'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=cool_square_amplitude_coll_single)
AOM_single_cooler.operations['cool_continuous_imag'] = cool_continuous_imag = SquarePulse(length=round(continuous_time/4)*4, amplitude=cool_square_amplitude_imag_single)

AOM_double_repumper.operations['repump_high_pulse_coll'] = SquarePulse(length=round(rc_high_time*u.s/4)*4, amplitude=repump_square_amplitude_coll_double)
AOM_double_repumper.operations['repump_high_pulse_imag'] = SquarePulse(length=round(rc_high_time*u.s/4)*4, amplitude=repump_square_amplitude_imag_double)
AOM_double_repumper.operations['repump_low_pulse'] = SquarePulse(length=round(rc_low_time*u.s/4)*4, amplitude=0)
AOM_double_cooler.operations['cool_high_pulse_coll'] = SquarePulse(length=round(rc_high_time*u.s/4)*4, amplitude=cool_square_amplitude_coll_double)
AOM_double_cooler.operations['cool_high_pulse_imag'] = SquarePulse(length=round(rc_high_time*u.s/4)*4, amplitude=cool_square_amplitude_imag_double)
AOM_double_cooler.operations['cool_low_pulse'] = cool_low_pulse = SquarePulse(length=round(rc_low_time*u.s/4)*4, amplitude=0)

AOM_tweez_mod.operations['tweez_step_pulse'] = tweez_step_pulse = SquarePulse(length=round(tweez_ramp_step_time/4)*4, amplitude=tweez_ramp_amplitude) 
AOM_tweez_mod.operations['tweez_plateau_pulse'] = SquarePulse(length=100, amplitude=tweez_plateau_amplitude) # this value is overridden by the time/slice value
AOM_tweez_mod.operations['tweez_high_pulse_coll'] = SquarePulse(length=round(tweez_high_time*u.s/4)*4, amplitude=tweez_square_amplitude_coll)
AOM_tweez_mod.operations['tweez_high_pulse_imag'] = SquarePulse(length=round(tweez_high_time*u.s/4)*4, amplitude=tweez_square_amplitude_imag)
AOM_tweez_mod.operations['tweez_low_pulse'] = SquarePulse(length=round(tweez_low_time*u.s/4)*4, amplitude=0)
AOM_tweez_spots1.operations['tweez_continuous'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=spot_amplitude1)
AOM_tweez_spots2.operations['tweez_continuous'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=spot_amplitude2)

In [ ]:
# for debug
machine.channels['AOM_debug_repumper'] = AOM_debug_repumper = SingleChannel(opx_output=('con1', 7), intermediate_frequency=0)
machine.channels['AOM_debug_tweez_spots1'] = AOM_debug_tweez_spots1 = SingleChannel(opx_output=('con1', 8), intermediate_frequency=tweez_high_freq)
machine.channels['AOM_debug_tweez_spots2'] = AOM_debug_tweez_spots2 = SingleChannel(opx_output=('con1', 8), intermediate_frequency=tweez_low_freq)


AOM_debug_repumper.operations['repump_high_pulse_imag'] = SquarePulse(length=round(rc_high_time*u.s/4)*4, amplitude=repump_square_amplitude_imag_double)
AOM_debug_repumper.operations['repump_low_pulse'] = SquarePulse(length=round(rc_low_time*u.s/4)*4, amplitude=0)
AOM_debug_tweez_spots1.operations['tweez_continuous'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=spot_amplitude1)
AOM_debug_tweez_spots2.operations['tweez_continuous'] = SquarePulse(length=round(continuous_time/4)*4, amplitude=spot_amplitude2)

In [7]:
AOM_double_cooler.operations['cool_high_pulse_imag'].amplitude

0.38

In [8]:
qua_config = machine.generate_config()

Write the QUA program

#### Example of the use of input streams
Input streams allow passing data from the client computer to a *running* job (QUA program). This might be useful during the optimization of parameters such as the *detuning* (which is controlled using update_frequency_) and possibly the *amplitude* (in this case I'm not sure actually, because it has to be multiplied by amp(v), which can take values in [-2,2]). See the guide for dynamical variation of the amplitude. 

The fundamental commands are:\
`declare_input_stream()`: declare a qua variable or vector to be used as an input stream to the qua program (is inside the program). Has all the params of a qua variable.\
`push_to_input_stream()`: used outside the program, is used to pass value from the job api to the qua program. It is a method of the job object.\
`advance_input_stream()`: inside the program, is used to access the next available data. *NB: if there is no available data, this command will pause OPX and wait until next data is available! It is critical were to put it!*

E.g.
```python 
with program() as example_input_stream:
    truth_table = declare_input_stream(bool, name='truth_table_input_stream', size=10)
    tau = declare_input_stream(int, name='tau_input_stream')
    ...
    with while_(some_qua_cond):
        advance_input_stream(tau)
        advance_input_stream(truth_table)
        play('operation', 'element0', duration=tau, condition=truth_table[0])
        play('operation', 'element1', duration=tau, condition=truth_table[1])
        play('operation', 'element2', duration=tau, condition=truth_table[2])
        ...
    ...

...
job = qm.execute(example_input_stream)
...
while some_cond:
    qubit_states = read_from_external_device()
    calc_tau = calculate_tau()
    job.push_to_input_stream('truth_table_input_stream', qubit_states)
    job.push_to_input_stream('tau_input_stream', calc_tau)
```

In [10]:
with program() as det_optimization_streams:
    input_cool_double_rf = declare_input_stream(int, name='cool_double_rf_input_stream') # NB: check the type of the input
    input_repump_double_rf = declare_input_stream(int, name='repump_double_rf_input_stream')

    with infinite_loop_():

        AOM_tweez_spots1.play("tweez_continuous")
        AOM_tweez_spots2.play("tweez_continuous")
        
    with infinite_loop_():
        # might stop the whole program when it returns to a new loop if no data is passed. See more carefully the documentation
        # might be a problem for the single pass?
        advance_input_stream(input_cool_double_rf)
        advance_input_stream(input_repump_double_rf)
        assign(IO1, input_cool_double_rf)
        assign(IO2, input_repump_double_rf)
        
        update_frequency('AOM_double_cooler', 200e6, keep_phase=True)
        update_frequency('AOM_double_repumper', 200e6, keep_phase=True)

        wait_for_trigger('AOM_tweez_mod') 
        wait_for_trigger('AOM_double_repumper')
        wait_for_trigger('AOM_double_cooler')
        wait_for_trigger('AOM_single_repumper')
        wait_for_trigger('AOM_single_cooler')

        # long_pulse('tweez_continuous', 'AOM_tweez_spots1', continuous_time_long_pulse)
        # long_pulse('tweez_continuous', 'AOM_tweez_spots2', continuous_time_long_pulse)

        # atoms loading
        long_ramp('tweez_step_pulse', 'AOM_tweez_mod', tweez_ramp_time)
        long_pulse('tweez_plateau_pulse', 'AOM_tweez_mod', tweez_plateau_time)
        align('AOM_tweez_mod', 'AOM_double_cooler', 'AOM_double_repumper','AOM_single_cooler','AOM_single_repumper','AOM_debug_repumper')

        # gap1
        play('tweez_step_pulse', 'AOM_tweez_mod', duration=round(gap1_time/4))
        play('cool_low_pulse', 'AOM_double_cooler', duration=round(gap1_time/4))
        play('repump_low_pulse', 'AOM_double_repumper', duration=round(gap1_time/4))
        play('repump_low_pulse', 'AOM_debug_repumper', duration=round(gap1_time/4))

        # long_pulse('cool_continuous_coll', 'AOM_single_cooler', gap1_time)
        # long_pulse('repump_continuous_coll', 'AOM_single_repumper', gap1_time)

        long_pulse('cool_continuous_coll', 'AOM_single_cooler', gap1_time + coll_time) # try to change
        long_pulse('repump_continuous_coll', 'AOM_single_repumper', gap1_time + coll_time)

        # collision
        stroboscopic(('tweez_low_pulse','tweez_high_pulse_coll'), 'AOM_tweez_mod', number_of_periods_collision)
        stroboscopic(('cool_high_pulse_coll','cool_low_pulse'), 'AOM_double_cooler', number_of_periods_collision, rc_delay_time)
        stroboscopic(('repump_high_pulse_coll','repump_low_pulse'), 'AOM_double_repumper', number_of_periods_collision, rc_delay_time)
        stroboscopic(('repump_high_pulse_imag','repump_low_pulse'), 'AOM_debug_repumper', number_of_periods_collision, rc_delay_time)

        update_frequency('AOM_double_cooler', input_cool_double_rf, keep_phase=True)
        update_frequency('AOM_double_repumper', input_repump_double_rf, keep_phase=True)

        # gap2
        play('tweez_high_pulse_imag', 'AOM_tweez_mod', duration=round(gap2_time/4))
        play('cool_low_pulse', 'AOM_double_cooler', duration=round(gap2_time/4))
        play('repump_low_pulse', 'AOM_double_repumper', duration=round(gap2_time/4))
        play('repump_low_pulse', 'AOM_debug_repumper', duration=round(gap2_time/4))

        long_pulse('cool_continuous_imag', 'AOM_single_cooler', gap2_time + imag_time) # try to change
        long_pulse('repump_continuous_imag', 'AOM_single_repumper', gap2_time + imag_time)

        # imaging
        stroboscopic(('tweez_low_pulse','tweez_high_pulse_imag'), 'AOM_tweez_mod', number_of_periods_imaging)
        stroboscopic(('repump_high_pulse_imag','repump_low_pulse'), 'AOM_double_repumper', number_of_periods_imaging, rc_delay_time)
        stroboscopic(('cool_high_pulse_imag','cool_low_pulse'), 'AOM_double_cooler', number_of_periods_imaging, rc_delay_time)
        stroboscopic(('repump_high_pulse_imag','repump_low_pulse'), 'AOM_debug_repumper', number_of_periods_imaging, rc_delay_time)

        # delay for image
        long_pulse('tweez_high_pulse_imag', 'AOM_tweez_mod', delay_imaging_time)
        long_pulse('cool_low_pulse', 'AOM_double_cooler', delay_imaging_time)
        long_pulse('repump_low_pulse', 'AOM_double_repumper', delay_imaging_time)
        long_pulse('repump_low_pulse', 'AOM_debug_repumper', delay_imaging_time)

        long_pulse('cool_continuous_imag', 'AOM_single_cooler', delay_imaging_time + imag_time) # try to change
        long_pulse('repump_continuous_imag', 'AOM_single_repumper', delay_imaging_time + imag_time)
        
        #align('AOM_tweez_mod', 'AOM_double_cooler', 'AOM_double_repumper', 'AOM_single_cooler', 'AOM_single_repumper')
        
        # imaging
        stroboscopic(('tweez_low_pulse','tweez_high_pulse_imag'), 'AOM_tweez_mod', number_of_periods_imaging)
        stroboscopic(('cool_high_pulse_imag','cool_low_pulse'), 'AOM_double_cooler', number_of_periods_imaging, rc_delay_time)
        stroboscopic(('repump_high_pulse_imag','repump_low_pulse'), 'AOM_double_repumper', number_of_periods_imaging, rc_delay_time)
        stroboscopic(('repump_high_pulse_imag','repump_low_pulse'), 'AOM_debug_repumper', number_of_periods_imaging, rc_delay_time)
        
        pause()

In [11]:
class StreamsHandler(PatternMatchingEventHandler):
    def __init__(self, patterns, qm, opx_job):
        super().__init__(patterns=patterns, ignore_directories=True)
        self.qm = qm
        self.job = opx_job

    def on_created(self, event):
        # this functoin is called whenever a new "pattern" file is created
        
        clear_output(wait=True)
        print(f"Processing file: {event.src_path}")
        time.sleep(0.2) # wait for M-LOOP to write the file
        with open(event.src_path, 'r') as file:
            line = file.readline().strip()
            new_parameters = np.array(eval(line.split('=')[1].strip())) # take the argument after the "=" and evaluate the string as python variable
        print("The Dog has watched:", new_parameters) # for debugging

        cool_double_rf_imag, repump_double_rf_imag = new_parameters
        self.job.push_to_input_stream('cool_double_rf_input_stream', int(cool_double_rf_imag))
        self.job.push_to_input_stream('repump_double_rf_input_stream', int(repump_double_rf_imag))
        print("New cooler frequency:", self.qm.get_io1_value())
        print("New repumper frequency:", self.qm.get_io2_value())
        self.job.resume()
        
        os.remove(event.src_path)
        #print("New values:")
        #print(f" cooler frequency: {cool_double_rf_imag}")
        #print(f" repumper frequency: {repump_double_rf_imag}")

In [12]:
# get the parameters from the exp_input file 
path_to_watch = r"C:\Users\EQM\Giovanni\OPXsetup\OPX_EQM\Optimization\imaging_parameters"
pattern = 'exp_input.txt'

if __name__ == '__main__':
    qm = qmm.open_qm(qua_config)
    job = qm.execute(det_optimization_streams)
    observer = Observer()
    event_handler = StreamsHandler(patterns=[pattern], qm=qm, opx_job = job)
    observer.schedule(event_handler, path=path_to_watch, recursive=True)
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

Processing file: C:\Users\EQM\Giovanni\OPXsetup\OPX_EQM\Optimization\imaging_parameters\exp_input.txt
The Dog has watched: [2.01269805e+08 1.98421500e+08]
New cooler frequency: {'io_number': 1, 'int_value': 0, 'fixed_value': 0.0, 'boolean_value': False}
New repumper frequency: {'io_number': 2, 'int_value': 0, 'fixed_value': 0.0, 'boolean_value': False}


Look at the reurned dictionary. Why are the values wrong?

In [ ]:
qm.close()
# qm.compile(det_optimization)